In [91]:
# pandas data analysis library
import pandas as pd

# regular expressions library
import re

# this library allows us to read xlsx files
import xlrd

Match Cost Price Column with Price Column 

In [92]:
# read csv file containing webscraper output
df = pd.read_csv('../data/webscrapper_output.csv')

Split Dataframe into variance and no variance

In [93]:
# determines whether a product is a duplicate based on the product's name
variant_products = (df['productname'].duplicated(keep=False))

# '~' makes it so only false values are returned
no_variant_products = ~variant_products

In [94]:
# assign df to products with no variance
df = df[no_variant_products]

Add Domain To Image Links

In [95]:
# this function adds website domain to image links
def add_domain_to_image_links(image_link):

    domain = 'www.wonatrading.com/'

    # if image link is a string add domain
    if isinstance(image_link, str):
        return domain + image_link

    # otherwise return 
    return ''

In [96]:
# apply add_domain_to_image_links to both image columns 
df['image1'] = df['image1'].apply(add_domain_to_image_links)
df['image2'] = df['image2'].apply(add_domain_to_image_links)

Clean Price Column

In [97]:
# this function returns the first price found in price column
def clean_price(price):

    pattern = re.compile(r'\d.{2}\d')  # pattern we're looking for

    matches = pattern.finditer(price)  # number of matches found

    matches_list = [match.group() for match in matches]

    return matches_list[0]

In [98]:
df['price'] = df['price'].apply(clean_price) 

Combine all 3 category columns into 1 Column

In [99]:
# combine each category column into 1
# JEWELRY;ANKLET/FASHION
df['category'] = df['category1'] + ';' + df['category2'] + '/' + df['category3']

# delete old category columns
df.drop(['category1', 'category2', 'category3'], axis=1, inplace=True)

Clean Description Functions

In [100]:
def remove_name_from_product_description(description):

    # r'\s' + adds a white space to the begining of the pattern
    pattern = r'\s' + df['productname']  # pattern we're looking for

    replacement = ''  # what to replace the string with

    target = description  # the string we want to replace

    # re.sub replaces the string and ignorescase
    replaced = re.sub(pattern, replacement, target, flags=re.IGNORECASE)

    # replace description of each product in original file
    return replaced

def remove_style_number_from_product_description(description):

    pattern = r'( Style No : )\d+\s' # pattern we're looking for

    replacement = ''  # replace pattern with nothing

    target = description  # the string we want to replace

    # re.sub replaces the string and ignorescase
    replaced = re.sub(pattern, replacement, target, flags=re.IGNORECASE)

    # return the cleaned string
    return replaced

def remove_colors_from_descrition(description):
    
    # pattern we're looking for
    # 2 patterns
    # (Color : )\w+\s OR (Color : )\w+,\s*\w+\s*
    pattern = r'((Color : )\w+\s|(Color : )\w+,\s*\w+\s*)'    

    replacement = ''             # replace pattern with nothing
    target = description         # the string we want to replace

    # re.sub replaces the string and ignorescase
    replaced = re.sub(pattern, replacement, target, flags=re.IGNORECASE)

    return replaced # return the cleaned string

(Clean Description) Dataframe Description Column = result of all the functions

In [101]:
# remove style number from description
df['description'] = df['description'].apply(remove_style_number_from_product_description)

# remove colors from description
df['description'] = df['description'].apply(remove_colors_from_descrition)

# list of all product names
productname_list = df['productname'].tolist()

# remove product names from product description column
df['description'] = df['description'].str.replace('|'.join(productname_list), '')

# prints entire column width
pd.set_option('display.max_colwidth', 1000)

# print description column
df['description']

0               Theme : Pearl  Size : 0.25"H, 9" + 3" L  One Side Only Lead and Nickel Compliant 
1                        Size : 0.1" H, 8.25" H, 2.5" L  One Side Only Lead and Nickel Compliant 
8    Theme : Message  Size : 2.25" X 0.7", 8.25" + 3" L  One Side Only Lead and Nickel Compliant 
Name: description, dtype: object

Change column names to the column names in no variant template

In [102]:
# 2 lists containing the field names for input and outputs
input_columns = ['productname','category','image1','image2','description','price']
output_columns = ['Product Name', 'Category','Product Image File - 1','Product Image File - 2','Product Description','Price']

# keys are original column names, values are what they should be changed to
change_column_names_dictionary = dict(zip(input_columns, output_columns))

# this is the command changes column names in dataframe to column names that are in no variant template
# inplace=True means that it applies the change to original data frame
df.rename(columns=change_column_names_dictionary, inplace=True)

df

,Product Name,Product Image File - 1,Product Image File - 2,Product Description,Price,Category
0,Freshwater Pearl Disc Beaded Anklet,www.wonatrading.com/images/20200312/AK0001-@GD-NMLT1@025H-9_3L@479682@375@01.jpg,,"Theme : Pearl Size : 0.25""H, 9"" + 3"" L One Side Only Lead and Nickel Compliant",3.75,JEWELRY;ANKLET/FASHION
1,Bead Link Layered Anklet,www.wonatrading.com/images/20200123/AK0002-@GD-HEM@01H-825_25L@473115@375@01.jpg,,"Size : 0.1"" H, 8.25"" H, 2.5"" L One Side Only Lead and Nickel Compliant",3.75,JEWELRY;ANKLET/FASHION
8,Rhinestone Pave Boss Anklet,www.wonatrading.com/images/20191206/AK1503-@GD-CRY-BOSS@225X07-825_3L@467403@200@01.jpg,www.wonatrading.com/images/20191206/des_img/AK1503-@GD-CRY-BOSS@225X07-825_3L@467403@200@01@1.jpg,"Theme : Message Size : 2.25"" X 0.7"", 8.25"" + 3"" L One Side Only Lead and Nickel Compliant",2.00,JEWELRY;ANKLET/FASHION


Create Data Frame for No Variant Template

In [103]:
# open excel file
no_variant_excel_df = pd.read_excel('No Variant Template Excel Spreadsheet.xlsx')

# print file
no_variant_excel_df

,Field,NEW PRODUCT WITH NO VARIANT,EXAMPLE
0,Item Type,Fill the column with: product,Product
1,Product ID,Leave blank,NaN
2,Product Name,Fill with :product name,2PCS - HEART CHARM METAL LINK LAYERED ANKLETS
3,Product Type,Fill the column with :P,P
4,Product Code/SKU,Fill with product style number.,479533
...,...,...,...
73,GPS Item Group ID,Leave blank,NaN
74,GPS Category,Leave blank,NaN
75,GPS Enabled,Fill with N,N
76,Tax Provider Tax Code,Leave blank,NaN


Column Names

In [104]:
# Column names are assigned to field column of excel file
column_names = no_variant_excel_df['Field']

# print column names
column_names

0                 Item Type
1                Product ID
2              Product Name
3              Product Type
4          Product Code/SKU
              ...          
73        GPS Item Group ID
74             GPS Category
75              GPS Enabled
76    Tax Provider Tax Code
77    Product Custom Fields
Name: Field, Length: 78, dtype: object

Default Values

In [105]:
# default values are assigned to EXAMPLE column of excel file
# .fillna('') coverts NaN values to empty strings
# .tolist() ensures the series data type is converted to a list
default_values = no_variant_excel_df['EXAMPLE'].fillna('').tolist()

# print default values
default_values

['Product',
 '',
 '2PCS - HEART CHARM METAL LINK LAYERED ANKLETS',
 'P',
 479533,
 '',
 'BIJOUX PIECES',
 '2PCS - HEART CHARM METAL LINK LAYERED ANKLETS',
 'Right',
 '• Theme : Heart\n• Size : 0.5"H, 7.5" + 3" L\n• One Side Only\n• Lead and Nickel Compliant',
 6,
 3,
 0,
 0,
 0,
 'N',
 '',
 0,
 0,
 0,
 0,
 'Y',
 'Y',
 'Y',
 'by product',
 50,
 10,
 'JEWELRY;ANKLET/FASHION',
 '',
 'https://www.wonatrading.com/images/20200311/AK0071-@GD-XX@2P-05H-75_3L@479533@300@01.jpg',
 '',
 'Y',
 0,
 '',
 'https://www.wonatrading.com/images/20200311/des_img/AK0071-@GD-XX@2P-05H-75_3L@479533@300@01@1.jpg',
 '',
 'N',
 1,
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'New',
 'N',
 0,
 'N',
 '',
 'N',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'N',
 '',
 'Color=Gold']

No Variant Template Dataframe

In [106]:
# DataFrame for 'No Variant Template.xlsx'
no_variant_template_df = pd.DataFrame(
    columns=column_names    # assign columns of dataframe to columns from excel file 
    )

# print dataframe
no_variant_template_df

Field,Item Type,Product ID,Product Name,Product Type,Product Code/SKU,Bin Picking Number,Brand Name,Option Set,Option Set Align,Product Description,...,GPS Age Group,GPS Color,GPS Size,GPS Material,GPS Pattern,GPS Item Group ID,GPS Category,GPS Enabled,Tax Provider Tax Code,Product Custom Fields


Add Data to no_variant_template_df

In [107]:
# append webscrapper data to no_variant_template_df
no_variant_template_df = no_variant_template_df.append(df, sort=False)

Fill NaN values with default values

In [108]:
no_variant_template_df.fillna(
    dict( # Dictionary of 2 lists
        zip(
            no_variant_template_df.columns, # keys are the column names
            default_values                  # values for each column are default_values list 
        )
    ),
    inplace=True # make change to original dataframe
)

# print dataframe
no_variant_template_df

,Item Type,Product ID,Product Name,Product Type,Product Code/SKU,Bin Picking Number,Brand Name,Option Set,Option Set Align,Product Description,...,GPS Age Group,GPS Color,GPS Size,GPS Material,GPS Pattern,GPS Item Group ID,GPS Category,GPS Enabled,Tax Provider Tax Code,Product Custom Fields
0,Product,,Freshwater Pearl Disc Beaded Anklet,P,479533,,BIJOUX PIECES,2PCS - HEART CHARM METAL LINK LAYERED ANKLETS,Right,"Theme : Pearl Size : 0.25""H, 9"" + 3"" L One Side Only Lead and Nickel Compliant",...,,,,,,,,N,,Color=Gold
1,Product,,Bead Link Layered Anklet,P,479533,,BIJOUX PIECES,2PCS - HEART CHARM METAL LINK LAYERED ANKLETS,Right,"Size : 0.1"" H, 8.25"" H, 2.5"" L One Side Only Lead and Nickel Compliant",...,,,,,,,,N,,Color=Gold
8,Product,,Rhinestone Pave Boss Anklet,P,479533,,BIJOUX PIECES,2PCS - HEART CHARM METAL LINK LAYERED ANKLETS,Right,"Theme : Message Size : 2.25"" X 0.7"", 8.25"" + 3"" L One Side Only Lead and Nickel Compliant",...,,,,,,,,N,,Color=Gold


In [109]:
# cost price column = price column
no_variant_template_df['Cost Price'] = no_variant_template_df['Price']

# print cost price column
no_variant_template_df['Cost Price']

0    3.75
1    3.75
8    2.00
Name: Cost Price, dtype: object